In [ ]:
# Import neccessary libraries for corner detection
import numpy as np
from PIL import Image
import cv2

In [ ]:
# View the image and resize it
imgs = []
img = cv2.imread('./images/raw/catan00.jpg')
imgs.append(img)
img = cv2.imread('./images/raw/catan01.jpg')
imgs.append(img)
for i in range(len(imgs)):
    imgs[i] = cv2.resize(imgs[i], (640, 640))


In [ ]:
# Define a show image function
def showImage(img):
    cv2.imshow('Frame', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# Convert images to greyscale
gray_imgs = []
for i in range(len(imgs)):
    gray_imgs.append(cv2.cvtColor(imgs[i], cv2.COLOR_BGR2GRAY))

In [ ]:
# Apply thresholding and find contours
ret, thrash = cv2.threshold(gray_imgs[0], 240, 255, cv2.CHAIN_APPROX_NONE)
contours, _ = cv2.findContours(thrash, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [9]:
# Iterate through the range of contours
MAX_HEXAGONS = 19
found_hexes = 0
for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)
    cv2.drawContours(gray_imgs[0], [approx], 0, (0,0,0), 5)
    x = approx.ravel()[0]
    y = approx.ravel()[1] - 5
    if len(approx) == 6:
        cv2.putText(gray_imgs[0], "Hexagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
        found_hexes += 1
    if found_hexes >= MAX_HEXAGONS:
        break

In [10]:
# Show the image and close the window
showImage(gray_imgs[0])

In [ ]:
# Apply a mask to isolate the board

# HEIGHT = 25
# SPACE_FROM_TOP = 140
# ROW_LENGTH = len(gray_img[0])
# COL_LENGTH = len(gray_img)
# # Isolate the top-most corners of the image
# for i in range(SPACE_FROM_TOP):
#     for j in range(ROW_LENGTH):
#         gray_img[i, j] = 0

# for i in range(SPACE_FROM_TOP+HEIGHT, COL_LENGTH):
#     for j in range(ROW_LENGTH):
#         gray_img[i, j] = 0


# showImage(gray_img)

In [ ]:
# Detect 'N' corners using goodFeaturesToTrack
N = 18
for i in range(len(imgs)):
    corners = cv2.goodFeaturesToTrack(
        imgs[i], # Source image
        N, # Number of corners to detect
        0.01, # Quality of corners (minimum confidence)
        20 # Minimum euclidian distance between corners
    )
    # Convert corners to integers instead of floats
    corners = np.int0(corners)
    # Draw corners
    for corner in corners:
        x, y = corner.ravel() # Flatten the array [[x, y]] -> [x, y] (torch squeeze)
        cv2.circle(imgs[i], (x, y), 5, (255, 0, 0), -1)

In [ ]:
showImage(imgs[0])
showImage(imgs[1])

In [ ]:
import torch
torch.__version__, torch.cuda.is_available()